In [1]:
%load_ext autoreload
%autoreload 2


In [2]:
cd ~/demres

/Users/zurfarosa/demres


In [3]:
import os
import sys

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.tools.tools import add_constant
from scipy.stats import chi2_contingency
from scipy import stats
import pylab as pl
from datetime import date, timedelta

import demres
from demres.common.constants import entry_type
from demres.demins.constants import Study_Design as sd
from demres.common import codelists
from demres.common.helper_functions import *
from demres.demins.statistical_functions import *

In [4]:
pd.set_option('display.max_columns', None)

In [5]:
subtype = 'all_dementia' # options: 'alzheimers', 'vascular','all_dementia'

In [6]:
window = '10_to_5' #options: '12_to_7','10_to_5','8_to_3'

In [7]:
pt_features = pd.read_csv('data/pt_data/processed_data/pt_features_demins_'+subtype+'_'+window+'.csv',delimiter=',',parse_dates=['index_date','data_end','data_start'],infer_datetime_format=True )

In [8]:
pt_features

,patid,yob,pracid,male,index_date,isCase,final dementia medcode,data_start,data_end,matchid,age_at_index_date,insomnia,insomnia_consultations,stroke,intellectual_disability,CHD_heart_failure_and_peripheral_vascular_disease,hypertension,diabetes,clin_sig_alcohol_use,mental_illness_non_smi,mental_illness_smi,sleep_apnoea,current_smoker,chronic_pulmonary_disease,CKD,mood_stabilisers_100_pdds,benzo_and_z_drugs_100_pdds,benzo_and_z_drugs<1096,benzo_and_z_drugs>1096,benzo_and_z_drugs_never_used,other_sedatives_100_pdds,fgas_100_pdds,sgas_100_pdds,sga_depots_100_pdds,fga_depots_100_pdds,antidepressants_100_pdds,non_insomnia_GP_consultations
0,6807236,30,236,1,2006-11-06,True,26270.0,1996-11-05,2006-11-17,35945,76,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.000000,0.000000,0,0,1,0.000000,0.000000,0.000000,0.0,0.0,0.000000,7
1,2275105,27,105,0,2006-04-25,True,1917.0,1996-04-26,2006-05-09,14685,79,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.000000,0.000000,0,0,1,0.000000,0.000000,0.000000,0.0,0.0,0.000000,53
2,10665377,20,377,0,2011-04-07,True,4693.0,2001-04-01,2011-04-14,55935,91,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.000000,0.000000,0,0,1,0.000000,0.000000,0.000000,0.0,0.0,0.000000,187
3,12440330,25,330,1,2010-03-30,True,4693.0,2000-03-22,2010-04-09,50156,85,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0.000000,1.055262,1,0,0,0.000000,0.134434,0.000000,0.0,0.0,1.491983,170
4,16055443,27,443,1,2010-12-20,True,1350.0,2000-12-15,2011-01-13,65609,83,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.000000,0.000000,0,0,1,0.000000,0.000000,0.000000,0.0,0.0,0.000000,215
5,2994148,19,148,1,2007-02-15,True,1916.0,1997-01-20,2007-02-21,21680,88,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0.000000,13.190775,0,1,0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,58
6,4552183,39,183,0,2010-09-29,True,26270.0,2000-09-22,2010-10-28,27821,71,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.000000,0.000000,0,0,1,0.000000,0.000000,0.000000,0.0,0.0,0.000000,5
7,6726262,33,262,0,2007-02-23,True,9509.0,1997-02-09,2007-03-23,40437,74,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.000000,0.000000,0,0,1,0.000000,0.000000,0.000000,0.0,0.0,0.000000,240
8,706017,25,17,1,2008-05-22,True,4693.0,1998-04-18,2008-06-05,2566,83,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0.000000,0.000000,0,0,1,0.000000,0.000000,0.000000,0.0,0.0,0.000000,215
9,14778043,21,43,0,2006-01-31,True,1350.0,1996-01-05,2006-02-23,6184,85,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.000000,0.000000,0,0,1,0.000000,0.000000,0.000000,0.0,0.0,0.000000,24


In [9]:
pt_features.columns

Index(['patid', 'yob', 'pracid', 'male', 'index_date', 'isCase',
       'final dementia medcode', 'data_start', 'data_end', 'matchid',
       'age_at_index_date', 'insomnia', 'insomnia_consultations', 'stroke',
       'intellectual_disability',
       'CHD_heart_failure_and_peripheral_vascular_disease', 'hypertension',
       'diabetes', 'clin_sig_alcohol_use', 'mental_illness_non_smi',
       'mental_illness_smi', 'sleep_apnoea', 'current_smoker',
       'chronic_pulmonary_disease', 'CKD', 'mood_stabilisers_100_pdds',
       'benzo_and_z_drugs_100_pdds', 'benzo_and_z_drugs<1096',
       'benzo_and_z_drugs>1096', 'benzo_and_z_drugs_never_used',
       'other_sedatives_100_pdds', 'fgas_100_pdds', 'sgas_100_pdds',
       'sga_depots_100_pdds', 'fga_depots_100_pdds',
       'antidepressants_100_pdds', 'non_insomnia_GP_consultations'],
      dtype='object')

In [10]:
characteristics = [column for column in pt_features.columns if column not in [
    'patid',  'yob', 'pracid', 'index_date', 'isCase',
    'final dementia medcode', 'data_start', 'data_end', 'matchid']]

In [11]:
baseline_characteristics = pd.DataFrame(index=characteristics,columns=['Cases','Controls','P value'])

In [12]:
def add_baseline_characteristics(baseline_df,variables):
    for variable in variables:        
        if set(pt_features[variable])=={0,1}: #if it is a boolean 1 or 0 variable
            positive_cases = pt_features.loc[(pt_features[variable]==1)&(pt_features['isCase']==1)]
            negative_cases = pt_features.loc[(pt_features[variable]==0)&(pt_features['isCase']==1)]
            positive_controls = pt_features.loc[(pt_features[variable]==1)&(pt_features['isCase']==0)]
            negative_controls = pt_features.loc[(pt_features[variable]==0)&(pt_features['isCase']==0)]
            baseline_df.loc[variable,'Cases'] = "{0:.1f}".format(len(positive_cases))
            baseline_df.loc[variable,'Controls'] = "{0:.1f}".format(len(positive_controls))
            baseline_df.loc[variable,'test']='chi2'
            obs = np.array([[len(positive_cases),len(negative_cases)],[len(positive_controls),len(negative_controls)]])
            chi2, p, dof, ex = chi2_contingency(obs, correction=False)
            baseline_df.loc[variable,'P value'] =  "{0:.3f}".format(p)
        else: #if it is a continuous variable
            cases = pt_features.loc[pt_features['isCase']==1,variable].values
            controls = pt_features.loc[pt_features['isCase']==0,variable].values
            baseline_df.loc[variable,'Cases'] = "{0:.2f}".format(np.mean(cases))
            baseline_df.loc[variable,'Controls'] = "{0:.2f}".format(np.mean(controls))
            baseline_df.loc[variable,'test']='2-sample-t-test'
            t_stat,p = stats.ttest_ind(cases,controls)
            baseline_df.loc[variable,'P value'] = "{0:.3f}".format(p)
        if variable in ['male','age_at_index_date']:
            baseline_df.loc[variable,'test']='(matched)'
#             baseline_df.loc[variable,'P value']=np.nan
    return baseline_df

In [13]:
add_baseline_characteristics(baseline_characteristics,characteristics).sort_values(by='test')

,Cases,Controls,P value,test
male,5138.0,5138.0,1.000,(matched)
age_at_index_date,84.67,84.67,1.000,(matched)
fga_depots_100_pdds,0.00,0.00,0.140,2-sample-t-test
sga_depots_100_pdds,0.00,0.00,nan,2-sample-t-test
sgas_100_pdds,0.03,0.02,0.115,2-sample-t-test
fgas_100_pdds,0.19,0.13,0.034,2-sample-t-test
other_sedatives_100_pdds,0.09,0.11,0.506,2-sample-t-test
benzo_and_z_drugs_100_pdds,0.76,0.53,0.000,2-sample-t-test
mood_stabilisers_100_pdds,0.28,0.16,0.004,2-sample-t-test
antidepressants_100_pdds,1.16,0.73,0.000,2-sample-t-test
